<a href="https://colab.research.google.com/github/cedamusk/fun-projects/blob/main/Cvs_And_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
!pip install torch torchvision transformers pillow


In [53]:
from IPython import get_ipython
from IPython.display import display

In [54]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from sklearn.model_selection import train_test_split
from google.colab import drive

In [55]:
from PIL import Image

In [56]:
def mount_google_drive():
  drive.mount('/content/drive')

In [57]:
def check_directory_contents(path):
  for item in os.listdir(path):
    print(item)

In [58]:
mount_google_drive()
dataset_path='/content/drive/MyDrive/minet'
check_directory_contents(dataset_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
biotite
pyrite
muscovite
bornite
malachite
quartz
chrysocolla


In [59]:
minerals=['biotite', 'pyrite', 'muscovite', 'bornite', 'malachite', 'quartz', 'chrysocolla']

In [60]:
image_paths=[]
labels=[]
for idx, mineral in enumerate(minerals):
  mineral_dir=os.path.join("/content/drive/MyDrive/minet", mineral)
  if os.path.exists(mineral_dir):
    for img_name in os.listdir(mineral_dir):
      if os.path.isfile(os.path.join(mineral_dir, img_name)):
        image_paths.append(os.path.join(mineral_dir, img_name))
        labels.append(idx)

In [61]:
train_paths, val_paths, train_labels, val_labels=train_test_split(image_paths, labels, test_size=0.2, random_state=42)

In [62]:
class MineralDataset(Dataset):
  def __init__(self, image_paths, labels, transform=None):
    self.image_paths=image_paths
    self.labels=labels
    self.transform=transform

  def __len__(self):
    return len(self.image_paths)

  def __getitem__(self, idx):
    image=Image.open(self.image_paths[idx]).convert('RGB')
    label=self.labels[idx]
    if self.transform:
      image=self.transform(image)
    return image, label

In [63]:
transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [64]:
train_dataset=MineralDataset(train_paths, train_labels, transform=transform)
val_dataset=MineralDataset(val_paths, val_labels, transform=transform)

In [65]:
train_loader=DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=16)

In [66]:
model_name='google/vit-base-patch16-224-in21k'
feature_extractor=ViTFeatureExtractor.from_pretrained(model_name)
model=ViTForImageClassification.from_pretrained(
    model_name, num_labels=len(minerals)
)

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
device=torch.device('cuda' if torch.cuda.is_available()else 'cpu')
model.to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=1e-4)
criterion=torch.nn.CrossEntropyLoss()

In [ ]:
num_epochs=5
for epoch in range(num_epochs):
  model.train()
  running_loss=0.0
  for images, labels in train_loader:
    images, labels=images.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs=model(images).logits
    loss=criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  print(f'Epoch{epoch+1}/{num_epochs}, Loss:{running_loss/len(train_loader):.4f}')



Epoch1/5, Loss:1.2338


In [ ]:
model.eval()
